In [15]:
import json

with open('data/valid_vietnam_tourism.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data = data['data'][0]
data['paragraphs']

[{'context': 'Việt Nam, một viên ngọc quý của Đông Nam Á, nổi bật với vẻ đẹp tự nhiên hùng vĩ và di sản văn hóa phong phú. Từ những bãi biển tuyệt đẹp như Nha Trang và Đà Nẵng cho đến các khu rừng nguyên sinh ở Tây Nguyên, cảnh quan Việt Nam mang đến nhiều trải nghiệm đa dạng cho du khách. Hà Nội, thủ đô ngàn năm văn hiến, hấp dẫn với nét cổ kính và ẩm thực đường phố phong phú, trong khi TP. Hồ Chí Minh lại sôi động và hiện đại, nơi lưu giữ dấu ấn lịch sử và sự phát triển năng động.\n\nViệt Nam còn nổi tiếng với các di sản văn hóa thế giới như Vịnh Hạ Long, phố cổ Hội An và khu di tích Mỹ Sơn, mỗi địa điểm đều có câu chuyện riêng và giá trị lịch sử sâu sắc. Du khách có thể khám phá các lễ hội truyền thống, tìm hiểu về phong tục tập quán của các dân tộc thiểu số, hoặc tham gia vào các lớp học nấu ăn để trải nghiệm ẩm thực phong phú của đất nước.\n\nVới những người yêu thích phiêu lưu, Việt Nam cung cấp nhiều hoạt động thú vị như trekking, lặn biển và khám phá các hang động kỳ vĩ. Bên cạ

In [18]:
para = data['paragraphs']
extracted_para = []
for i in range(len(para)):
    extracted_para.append(para[i]['context'])
joined_para = '\n\n'.join(extracted_para)

with open('data/vietnam_tourism.txt', 'w', encoding='utf-8') as f:
    f.write(joined_para)

In [1]:
with open('tuyen quang.txt', 'r', encoding='utf-8') as f:
    data = f.read()
data

'Khám phá các địa điểm lịch sử nổi bật\nSở hữu tới 546 di tích, trong đó có 435 di tích lịch sử, Tuyên Quang được ví như một “bảo tàng cách mạng sống” – nơi lưu giữ những dấu ấn hào hùng của dân tộc Việt Nam qua từng giai đoạn lịch sử.\n\nKhu di tích lịch sử Tân Trào – Thủ đô khu giải phóng\nTọa lạc tại huyện Sơn Dương, khu di tích Tân Trào là quần thể di tích lịch sử cấp quốc gia đặc biệt, nơi Trung ương Đảng và Chủ tịch Hồ Chí Minh từng sống và làm việc, chuẩn bị cho Cách mạng Tháng Tám năm 1945. Đây là địa điểm không thể bỏ qua khi đến Tuyên Quang.\nCác điểm tham quan tiêu biểu trong khu di tích:\n- Đình Tân Trào: Đình Tân Trào là nơi diễn ra Quốc dân Đại hội ngày 16/8/1945, thông qua 10 chính sách lớn của Việt Minh và quyết định Tổng khởi nghĩa. Kiến trúc truyền thống kết hợp không gian yên bình khiến nơi đây trở thành điểm đến mang đậm tính biểu tượng.\n- Lán Nà Lừa: Nằm sâu trong rừng, Lán Nà Lừa là nơi Bác Hồ sống và làm việc từ cuối tháng 5 đến tháng 8 năm 1945. Công trình bằng

In [2]:
from chromadb import PersistentClient
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import os

# Initialize the embedding model

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# Initialize ChromaDB client
db_dir = 'db/tuyen_quang'
os.makedirs(db_dir, exist_ok=True)
chroma_client = PersistentClient(path=db_dir)

# # The text to process

# text = data

# # 1. Split text into words

# words = text.split()

# # 2. Create chunks of ~350 words

# chunk_size = 350
# chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
chunks = [chunk.strip() for chunk in data.split("\n\n\n") if chunk.strip()]

# 3. Embed each chunk using SentenceTransformer

embeddings = model.encode(chunks, convert_to_numpy=True)

# 4. Create or get a Chroma collection

collection_name = "tuyen_quang"
if collection_name in chroma_client.list_collections():
    collection = chroma_client.get_collection(collection_name)
else:
    collection = chroma_client.create_collection(name=collection_name, embedding_function=None)

# 5. Add chunks and embeddings to Chroma

collection.add(
documents=chunks,
embeddings=embeddings.tolist(),
ids=[f"chunk_{i}" for i in range(len(chunks))]
)

print(f"Added {len(chunks)} chunks to ChromaDB collection '{collection_name}'")



c:\Users\OS TDTS\anaconda3\envs\nlp_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Added 5 chunks to ChromaDB collection 'tuyen_quang'


In [7]:
# Đọc database từ đầu nè
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path=db_dir)
collection = client.get_collection(name='tuyen_quang')

# collection = hello['collection']
query = 'Tuyên Quang du lịch lịch sử có điểm nào hay?'

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
query_embed = model.encode(query, normalize_embeddings=True)

results=collection.query(
    query_embeddings=[query_embed],
    n_results=3,
    include=['metadatas', 'documents', 'distances']
)
print(results['ids'][0])
results['documents'][0]

['chunk_1', 'chunk_4', 'chunk_3']


['Du lịch văn hóa và tâm linh\nTuyên Quang không chỉ là vùng đất giàu truyền thống cách mạng mà còn sở hữu đời sống văn hóa – tâm linh phong phú, gắn liền với tín ngưỡng dân gian, phong tục tập quán và các lễ hội đặc sắc của cộng đồng các dân tộc thiểu số. Đây là một trong những điểm mạnh giúp Tuyên Quang ngày càng thu hút du khách yêu thích trải nghiệm bản địa và tìm hiểu chiều sâu văn hóa vùng cao.\n\nCác đền thờ linh thiêng tại Tuyên Quang\nĐền Hạ (đền Tam Kỳ), Đền Thượng, Đền Ỷ La: Ba ngôi đền nổi tiếng nằm tại thành phố Tuyên Quang, thường được ghép thành tuyến hành hương “Tam đền” thu hút đông đảo du khách mỗi dịp lễ Tết. Các đền này thờ Mẫu Thượng Ngàn, Mẫu Thoải và các vị thần bản địa, gắn liền với tín ngưỡng thờ Mẫu của người Việt. Không gian tĩnh lặng, linh thiêng kết hợp với kiến trúc cổ kính tạo nên điểm đến vừa mang tính tâm linh, vừa có giá trị văn hóa.\nĐền Pác Tạ và đền Pác Vãng (Na Hang): Nằm gần hồ Na Hang, hai ngôi đền này không chỉ là nơi sinh hoạt tín ngưỡng của ng

# Test chunking trên data mới

In [38]:
with open('data/texts/thua_thien_hue.txt', 'r', encoding='utf-8') as f:
    data = f.read()

data_heading_split = data.split('\n\n\n')
data_heading_split = [i for i in data_heading_split if len(i) > 0]
data_heading_split

['Tổng quan\nNằm trên dải đất miền Trung, Huế là thành phố di sản văn hóa thế giới. Cố đô vốn có lịch sử và truyền thống lâu đời với những giá trị và bản sắc độc đáo.',
 '\n\nHuế mùa nào đẹp?\nMùa du lịch Huế từ tháng 11 năm trước đến tháng 4 năm sau, đẹp nhất là 3-4 tháng đầu năm, tiết trời mát mẻ. Với những du khách yêu núi, biển, thích ngắm bình minh và hoàng hôn, thời điểm nên đi là tháng 6-8 khi mặt trời đẹp nhất trong năm, nước biển xanh nhất.\nHuế không hẳn có mùa khô, mà chỉ có mùa mưa và mùa ít mưa. Mùa ít mưa thường kéo dài từ tháng 3 đến tháng 8, trời nóng nực và oi bức. Khoảng tháng 9 đến tháng 1 năm sau là mùa mưa, trời lạnh. Đặc biệt, từ tháng 10 đến tháng 11 cũng chính là mùa bão, mưa to tưởng như không dứt.\nMùa xuân kéo dài từ cuối tháng 1 đến tháng 3, thời tiết lúc này rất đẹp, trời có nắng nhẹ, đôi khi se se lạnh. Từ tháng 4 đến tháng 7, mùa hè nắng nóng, Huế cũng là địa điểm du lịch lý tưởng với những du khách yêu thích biển. Cố đô vào hè có sắc vàng của hoa điệp, h

In [39]:
heading = []
for head in data_heading_split:
    head = head.strip('\n').split('\n')[0]
    if len(head) > 0:
        heading.append(head)    
    else:
        continue
print(heading)

['Tổng quan', 'Huế mùa nào đẹp?', 'Di chuyển', 'Khách sạn, resort', 'Khám phá', 'Ăn chơi']


In [40]:
for i in range(len(data_heading_split)):
    data_heading_split[i] = data_heading_split[i].replace(f'{heading[i]}\n', '')
    data_heading_split[i] = data_heading_split[i].strip('\n').strip()
    if len(data_heading_split[i].split()) > 500:
        data = data_heading_split[i].strip().split("\n\n")
        print(f'{heading[i]} long chunk:\n{data}\n----------\n')
    else:
        print(f'{heading[i]}:\n{data_heading_split[i]}\n----------\n')

Tổng quan:
Nằm trên dải đất miền Trung, Huế là thành phố di sản văn hóa thế giới. Cố đô vốn có lịch sử và truyền thống lâu đời với những giá trị và bản sắc độc đáo.
----------

Huế mùa nào đẹp?:
Mùa du lịch Huế từ tháng 11 năm trước đến tháng 4 năm sau, đẹp nhất là 3-4 tháng đầu năm, tiết trời mát mẻ. Với những du khách yêu núi, biển, thích ngắm bình minh và hoàng hôn, thời điểm nên đi là tháng 6-8 khi mặt trời đẹp nhất trong năm, nước biển xanh nhất.
Huế không hẳn có mùa khô, mà chỉ có mùa mưa và mùa ít mưa. Mùa ít mưa thường kéo dài từ tháng 3 đến tháng 8, trời nóng nực và oi bức. Khoảng tháng 9 đến tháng 1 năm sau là mùa mưa, trời lạnh. Đặc biệt, từ tháng 10 đến tháng 11 cũng chính là mùa bão, mưa to tưởng như không dứt.
Mùa xuân kéo dài từ cuối tháng 1 đến tháng 3, thời tiết lúc này rất đẹp, trời có nắng nhẹ, đôi khi se se lạnh. Từ tháng 4 đến tháng 7, mùa hè nắng nóng, Huế cũng là địa điểm du lịch lý tưởng với những du khách yêu thích biển. Cố đô vào hè có sắc vàng của hoa điệp, h

In [41]:
dulich = heading[4]
dulich_data = data_heading_split[4].split('\n\n')

for place in dulich_data:
    print(f'{dulich} - {place}')
    if len(place.split()) > 500:
        print('\n---TOO LONG---\n')

Khám phá - Phá Tam Giang
Hệ đầm phá Tam Giang - Cầu Hai có diện tích hơn 22.000 ha trải dài 68 km thuộc huyện Phong Điền, Quảng Điền, Phú Vang, Phú Lộc và thị xã Hương Trà. Đây là đầm phá nước lợ lớn nhất Đông Nam Á.
Để ngắm bình minh ở phá Tam Giang, bạn cần xuất phát lúc 5h30, di chuyển khoảng 45 phút cho quãng đường khoảng 20 km. Đừng quên thử đặc sản từ tôm cá, trong đó phải kể đến bánh khoái cá kình. Đây là loại cá được bắt từ dưới phá và bán tại khu chợ gần đó. Thịt cá kình rất ngọt ăn cùng bánh khoái và nước mắm mặn sẽ khiến du khách nhớ mãi.
Khám phá - Rừng ngập mặn Rú Chá
Rừng ngập mặn nguyên sinh thuộc phá Tam Giang, làng Thuận Hòa, xã Hương Phong, thị xã Hương Trà. "Rú" nghĩa là rừng, còn "chá" là cây chá ở vùng ngập mặn. Nếu đến đây vào mùa thu bạn còn được chiêm ngưỡng màu vàng rực của những bông chá nở khắp rừng.
Các bạn trẻ tới rừng Rú Chá rất thích thú chụp ảnh ở con đường bê tông sâu hun hút và hai hàng cây bao bọc. Từ trung tâm thành phố, bạn chạy theo quốc lộ 49 sẽ t

# Main code logic

In [78]:
import glob
import os

def extract_chunks_from_file(data_path):
    '''
    Extract thành các chunks cho từng tỉnh
    '''
    # Đọc data
    with open(data_path, 'r', encoding='utf-8') as f:
        data = f.read()
    
    # Trích xuất tên vùng
    region = os.path.splitext(os.path.basename(data_path))[0]

    # Tách các phần của data ra
    data_heading_split = data.split('\n\n\n')
    data_heading_split = [i for i in data_heading_split if len(i) > 0]

    # Trích xuất heading của các data
    heading = []
    for head in data_heading_split:
        head = head.strip('\n').split('\n')[0]
        if len(head) > 0:
            heading.append(head)    
        else:
            continue
    
    chunked_data = []
    # Chunk thành các data
    for i in range(len(data_heading_split)):
        # Trong trường hợp phần quá dài
        if len(data_heading_split[i].split()) > 500:
            data_heading_split[i] = data_heading_split[i].strip().split('\n\n')
            for j in range(1, len(data_heading_split[i])):
                data_heading_split[i][j] = region + '\n' + heading[i] + '\n' + data_heading_split[i][j]
                chunked_data.append(data_heading_split[i][j])
        else:
            chunked_data.append(region + '\n' + data_heading_split[i])
    
    return chunked_data

def extract_chunks(path_to_folder, general_file_path):
    '''
    Pipeline extract chunks cho toàn bộ file data trong folder
    '''
    # Trích xuất path tới các file
    folder = path_to_folder
    files = glob.glob(f'{folder}/*.txt')
    
    with open(general_file_path, 'r', encoding='utf-8') as f:
        general_file = f.read()

    # Lấy thông tin từ trong file general trước
    chunks = general_file.split('\n\n')

    # Trích xuất chunks từ các file vùng miền cá nhân
    for file in files:
        chunks += extract_chunks_from_file(file)

    return chunks

def main(path_to_folder, general_file_path):
    chunks = extract_chunks(path_to_folder, general_file_path)

path_to_folder = 'data/texts'
general_info = 'data/vietnam_tourism.txt'
chunks_new = extract_chunks(path_to_folder, general_info)

In [79]:
chunks_new == chunks

True